In [1]:
import folium 
import requests
import json
import pandas as pd

In [2]:
dataset = pd.read_csv("Proyek Akhir/Linkedin Job Search.csv")
dataset = dataset.drop(columns=["Unnamed: 15","Unnamed: 16"])
kota_list = ['Jakarta', 'Surabaya', 'Bandung', 'Cikarang', 'Denpasar', 'Batang', 'Yogyakarta', 'Malang',
              'Pontianak', 'Kendal', 'Muarajawa', 'Tangerang', 'Pekanbaru', 'Medan', 'Lampung', 'Kuta',
              'Ubud', 'Bali','Bekasi', "East Java"]

# Fungsi untuk mengganti lokasi dengan nama kota sesuai daftar kota
def ganti_kota(lokasi):
    if isinstance(lokasi, str):  # Memastikan bahwa nilai adalah string
        for kota in kota_list:
            if kota in lokasi:
                return kota
    return lokasi

df = dataset.copy()
# Mengganti lokasi yang mengandung kota dengan nama kota yang sesuai
df['Company Location'] = df['Company Location'].apply(ganti_kota)
df['Company Location'] = df["Company Location"].replace(to_replace="East Java", value="Jawa Timur")
df['Company Location'] = df["Company Location"].replace(to_replace="Lampung", value="Bandar Lampung")
df = pd.DataFrame(df)
df

,Job Title,Company Name,Location,URL,Company Location,company size,Industry_stats,Job Qualifications,Job Skills,Job Skills.1,Job Skills.2,Profile,Skills,Type,Work Hours
0,Python Machine Learning Developer,Demandlane,13 applicants,https://www.linkedin.com/jobs/search/?currentJ...,Indonesia,11-50 employees,Legal Services 11-50 employees 25 on LinkedIn,"AWS Lambda, Amazon Redshift, Amazon Web Servic...",1 school alum works here,5+ years of work experience with Machine Learning,"Skills: Pandas (Software), TensorFlow, +8 more",Demandlane,5+ years of work experience with Machine Learning,Remote,Full-time
1,Python Machine Learning Developer,Demandlane,13 applicants,https://www.linkedin.com/jobs/search/?currentJ...,Indonesia,11-50 employees,Legal Services 11-50 employees 25 on LinkedIn,"AWS Lambda, Amazon Redshift, Amazon Web Servic...",1 school alum works here,5+ years of work experience with Machine Learning,"Skills: Pandas (Software), TensorFlow, +8 more",Demandlane,5+ years of work experience with Machine Learning,Remote,Full-time
2,Computer Aided Design Engineer (U.S.-Based Cli...,Eterna Indonesia,13 applicants,https://www.linkedin.com/jobs/search/?currentJ...,Indonesia,51-200 employees,Outsourcing and Offshoring Consulting 51-200 ...,"3D Modeling, AutoCAD, Computer-Aided Design (C...","Skills: SketchUp, AutoCAD, +6 more","3D Modeling, AutoCAD, Computer-Aided Design (C...",Try Premium for IDR0,Eterna Indonesia,"3D Modeling, AutoCAD, Computer-Aided Design (C...",Remote,Full-time
3,"Software Engineer Full-time (AI, Video and Aud...",Videfly,13 applicants,https://www.linkedin.com/jobs/search/?currentJ...,Jakarta,2-10 employees,Software Development 2-10 employees 7 on Li...,"Amazon Web Services (AWS), Back-End Web Develo...","Skills: TypeScript, OpenGL, +8 more",1+ years of work experience with JavaScript,Try Premium for IDR0,Videfly,1+ years of work experience with JavaScript,Remote,Full-time
4,"Artificial Intelligence Engineer, 2 Hour Learn...",Crossover,13 applicants,https://www.linkedin.com/jobs/search/?currentJ...,Surabaya,"5,001-10,000 employees","IT Services and IT Consulting 5,001-10,000 em...","Angular, Artificial Intelligence (AI), Back-En...","Skills: Svelte, Flask, +8 more","Angular, Artificial Intelligence (AI), Back-En...",Try Premium for IDR0,Crossover,"Angular, Artificial Intelligence (AI), Back-En...",Remote,Full-time
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,Data Engineer Academic Instructor,Alterra Indonesia,4 applicants,https://www.linkedin.com/jobs/search/?currentJ...,Jakarta,201-500 employees,IT Services and IT Consulting 201-500 employe...,NaN,"Skills: Data Engineering, Presentations, +8 more","Communication, Computer Science, Data Engineer...",Try Premium for IDR0,Alterra Indonesia,"Communication, Computer Science, Data Engineer...",On-site,Full-time
260,Data Science Lead,Talent Insider,4 applicants,https://www.linkedin.com/jobs/search/?currentJ...,Jakarta,11-50 employees,"Technology, Information and Internet 11-50 em...",NaN,"Skills: Datasets, R (Programming Language), +8...","Analytical Skills, Analytics, Artificial Intel...",Try Premium for IDR0,Talent Insider,"Analytical Skills, Analytics, Artificial Intel...",On-site,Full-time
261,Senior Machine Learning,JULO,4 applicants,https://www.linkedin.com/jobs/search/?currentJ...,Jakarta,"501-1,000 employees","Financial Services 501-1,000 employees 812 ...",NaN,1 school alum works here,"Attention to Detail, Computer Vision, Credit A...","Skills: Machine Learning, Machine Learning Alg...",JULO,"Attention to Detail, Computer Vision, Credit A...",Hybrid,Full-time
262,Senior Data Analyst in MarTech (Relocation to ...,Bolt,17 applicants,https://www.linkedin.com/jobs/search/?currentJ...,Jakarta,"1,001-5,000 employees","Software Development 1,001-5,000 employees ...","A/B Testing, Business Insights, Data Analysis,...","Skills: Data Analytics, SQL, +7 more","A/B Testing, Business Insights, Data Analysis,...",Try Premium for IDR0,Bolt,"A/B Testing

In [3]:
with open("location.json",'r') as f:
    data = json.load(f)
 
data_geo = pd.DataFrame(data)
print(data_geo.head(5))

        Kota                                  Georeferenced
0    Jakarta  POINT (106.84634081531794 -6.196639809431919)
1    Bandung                   POINT (107.608800 -6.899350)
2   Surabaya                   POINT (112.740515 -7.244582)
3     Malang                           POINT (112.65 -7.98)
4  Tangerang                           POINT (106.65 -6.15)


In [4]:
df_jobs = df.merge(data_geo, left_on='Company Location', right_on='Kota', how='left')
df_jobs = pd.DataFrame(df_jobs)
df_jobs = df_jobs.drop(columns=["Company Location"], axis=1)
df_jobs = df_jobs.dropna(subset=["Kota"])
print(df_jobs)  

                                             Job Title       Company Name  \
0                    Python Machine Learning Developer         Demandlane   
1                    Python Machine Learning Developer         Demandlane   
2    Computer Aided Design Engineer (U.S.-Based Cli...   Eterna Indonesia   
3    Software Engineer Full-time (AI, Video and Aud...            Videfly   
4    Artificial Intelligence Engineer, 2 Hour Learn...          Crossover   
..                                                 ...                ...   
259                  Data Engineer Academic Instructor  Alterra Indonesia   
260                                  Data Science Lead     Talent Insider   
261                            Senior Machine Learning               JULO   
262  Senior Data Analyst in MarTech (Relocation to ...               Bolt   
263                      Business Analytics Supervisor             Danone   

          Location                                                URL  \
0 

In [5]:
records = []
keys = ("Kota","Company Name","Job Title","URL","Type","Work Hours","Georeferenced")

for index, row in df_jobs.iterrows():
    records.append({key: row[key] for key in keys})

print(records[0])

{'Kota': 'Indonesia', 'Company Name': 'Demandlane', 'Job Title': 'Python Machine Learning Developer', 'URL': 'https://www.linkedin.com/jobs/search/?currentJobId=4014358169&distance=25&geoId=102478259&keywords=Data%20Scientist&origin=JOB_SEARCH_PAGE_KEYWORD_HISTORY&refresh=true', 'Type': 'Remote', 'Work Hours': 'Full-time', 'Georeferenced': 'POINT (117.343188 -0.878596)'}


In [6]:
for record in records:
    coords = record["Georeferenced"]
    longitude, latitude = coords.split("(")[-1].split(")")[0].split()
    record["Longitude"] = float(longitude) 
    record["Latitude"] = float(latitude) 

records[0]

{'Kota': 'Indonesia',
 'Company Name': 'Demandlane',
 'Job Title': 'Python Machine Learning Developer',
 'URL': 'https://www.linkedin.com/jobs/search/?currentJobId=4014358169&distance=25&geoId=102478259&keywords=Data%20Scientist&origin=JOB_SEARCH_PAGE_KEYWORD_HISTORY&refresh=true',
 'Type': 'Remote',
 'Work Hours': 'Full-time',
 'Georeferenced': 'POINT (117.343188 -0.878596)',
 'Longitude': 117.343188,
 'Latitude': -0.878596}

In [22]:
records[0]["Job Title"]

'Python Machine Learning Developer'

In [37]:
map = folium.Map(location=[-0.789275, 113.921327],zoom_start=6)

# pekerjaan_table = f"""
#     <div style='width:200px'>
#         <h4>The Job</h4>
#         <p>{records['Job Title']}</p>
#     </div>
# """

# popup_content = f"<h3>{records['Kota']}</h3>{pekerjaan_table}"

# # for record in records:
# #     coords = (record['Latitude'], record['Longitude'])
# #     folium.Marker(coords, popup=record['Kota']).add_to(map)

# # folium.Marker([records[0]["Latitude"], records[0]["Longitude"]],
# #               popup="<h1> Surabaya <h1><p> Ini adalah pekerjaan yang ada di surabaya<p>",
# #               tooltip="Indonesia",
# #               icon=folium.Icon(icon="flag", icon_color="white", color="red")).add_to(map)

# # for record in records:
# #     pekerjaan_table = generate_table(record["Job Title"])
# #     popup_content = f"<h3>{record['Kota']}</h3>{pekerjaan_table}"
    
# folium.Marker([record["Latitude"], record["Longitude"]],
#                 popup=folium.Popup(popup_content, max_width=1000),
#                 tooltip=f"Lowongan di {record['Kota']}",
#                 icon=folium.Icon(icon="info-sign", color="blue")).add_to(map)
    
for record in records:
    # Membuat tabel pekerjaan horizontal untuk masing-masing kota
    pekerjaan_table = """
        <table style='width:100%; table-layout:fixed;'>
            <tr><th style='white-space:nowrap;'>Pekerjaan</th></tr>
            <tr><td style='white-space:normal; word-wrap:break-word;'>{}</td></tr>
        </table>
    """.format(record["Job Title"])
    
    # Konten pop-up dengan tabel pekerjaan
    popup_content = f"<h3>{record['Kota']}</h3>{pekerjaan_table}"
    
    # Menambahkan marker untuk setiap record
    folium.Marker(
        [record["Latitude"], record["Longitude"]],
        popup=folium.Popup(popup_content, max_width=250),  # Atur lebar pop-up
        tooltip=f"Lowongan di {record['Kota']}",
        icon=folium.Icon(icon="info-sign", color="blue")
    ).add_to(map)

map

In [8]:
from folium.plugins import FastMarkerCluster

map = folium.Map(location=[-0.789275, 113.921327],zoom_start=5)
latitudes = [a['Latitude'] for a in records]
longitudes = [a['Longitude'] for a in records]

FastMarkerCluster(data=list(zip(latitudes, longitudes))).add_to(map)

map